# A* AND BIDIRECTIONAL BFS

## Imports

These are all the imports needed to solve this task:

In [1]:
from heapq import heappop, heappush
from pocket_cube.cube import Cube
from pocket_cube.cube import Move
from tests import case1, case2, case3, case4
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

## Test cases

These are the test cases that will be used to run and compare the algorithms:

In [2]:
cube1 = Cube(case1)
cube2 = Cube(case2)
cube3 = Cube(case3)
cube4 = Cube(case4)

## A* algorithm

In order to implement the A* algorithm I used: <br>
- a priority queue to store all the possible routes generated from a position and their corresponding cost
- a hash map (dictionary) to store the parents and children. <br> 
#### The heuristic function:
The heuristic function calculates all the different colors compared to the goal state of the algorithm and then divides it by 8 (because every move will displace 8 colors from their original spot), making it an admissible function

In [6]:
def heuristic_score(cube):
        score = 0
        for i in range(len(cube.state)):
            if cube.goal_state[i] != cube.state[i]:
                score += 1
    
        return score / 8

### Some functions that will help solve the cube:

In [4]:
# compares the current state with the goal state

def is_solved_astar(cube):
    comparator = cube.goal_state == cube.state
    return comparator.all()

In [5]:
# generate all neighbours possible using all possible moves

def get_neighbours(cube):
    return [Cube.move_state(cube.clone_state(), move) for move in range(6)]

In [6]:
# the A* algorithn
def astar(cube):
    priority_queue = []
    heappush(priority_queue, (0 + heuristic_score(cube), cube.clone()))
    discovered = {cube.hash(): (None, 0)}
    
    while priority_queue:
        current_cost, current_node = heappop(priority_queue)
        current_node_hash =current_node.hash() 
        
        if is_solved_astar(current_node):
            print("solved cube!")
            break
        else:
            for neighbour in get_neighbours(current_node):
                # we iterate trough all the possible neighbours from this
                # current state
                probe_cube = Cube(scrambled=False)
                probe_cube.state = neighbour
                probe_cube_hash = probe_cube.hash()
                
                # calculate the cost
                new_cost = discovered[current_node_hash][1] + 1
                
                # if the state is not inside the hash map or its new cost is smaller
                # compared to what we have in the hash map, we add/update it
                if probe_cube_hash not in discovered or new_cost < discovered[probe_cube_hash][1]:
                    discovered[probe_cube_hash] = (current_node_hash, new_cost)
                    heappush(priority_queue, (new_cost + heuristic_score(probe_cube), probe_cube))
    
    
    
    solution = []
    current_node = current_node.hash()
    while current_node is not None:
        solution.append(current_node)
        current_node, _ = discovered[current_node]
    path_len = len(solution) - 1
    discovered_nodes = len(discovered)
    
    print(f"Total number of nodes discovered: {discovered_nodes}")
    print(f"Length of the solution is: {path_len}")
    print(f"Solution is:\n")
    solution = solution[::-1]
    for sol in solution:
        print(f"\t{sol}")
    
    return discovered_nodes, path_len
        

### Let's see how well this algorithm performs with the given heuristic

In [7]:
%%time
discovered_nodes_case1_astar, path_len_case1_astar = astar(cube1)

solved cube!
Total number of nodes discovered: 374
Length of the solution is: 5
Solution is:

	255011422234350044103135
	502511322234300144145035
	500111252232303444143550
	550021514242303440113352
	050511114242333340405252
	000011112222333344445555
CPU times: total: 15.6 ms
Wall time: 19.4 ms


In [8]:
%%time
discovered_nodes_case2_astar, path_len_case2_astar = astar(cube2)

solved cube!
Total number of nodes discovered: 5187
Length of the solution is: 7
Solution is:

	115544133204352340002215
	105043415224352340032115
	001513215224332040445315
	002113245220331540443551
	052121434200331540413255
	051521214243330040415352
	001111222233330044445555
	000011112222333344445555
CPU times: total: 266 ms
Wall time: 249 ms


In [9]:
%%time
discovered_nodes_case3_astar, path_len_case3_astar = astar(cube3)

solved cube!
Total number of nodes discovered: 73309
Length of the solution is: 9
Solution is:

	311401145205353242254003
	321514013205353240254104
	323214153201350540250441
	343111545201350542220043
	413301045201320242155543
	410401015202323342155354
	410101025233320442153455
	041111525233330442420055
	040411115252333342425050
	000011112222333344445555
CPU times: total: 4.03 s
Wall time: 4.03 s


In [10]:
%%time
discovered_nodes_case4_astar, path_len_case4_astar = astar(cube4)

solved cube!
Total number of nodes discovered: 836285
Length of the solution is: 11
Solution is:

	521252101210333445030445
	152202301210303445431545
	153402221230301045434155
	153322025210301043414554
	151022335202301043415445
	131123235242301040455540
	132323425210301140455054
	102442235200301143435155
	041212535200331443420155
	041412125253330043425051
	001111222233330044445555
	000011112222333344445555
CPU times: total: 52.8 s
Wall time: 52.7 s


## Bidirectional BFS

I will define a new function named **compared_states** that resembles the function **is_solved_astar**. Except from this, this algorithm also uses the **get_neighbours** function.

For this algorithm, I used 2 queues, one that will be used to store the visited nodes discovered when starting from the source (unsolved cube), the other one used to store the visited nodes discovered when starting from the destination (the goal/solved cube). <br>
When analyzing a node, if the node is not in the queue of the direction we came from (source / destination) we check to see if it is in the other's queue. If yes, we found our path and the problem is solved.

In [11]:
# compares the current state with the goal state

def compare_states(cube_src, cube_dest):
    comparator = cube_src.state == cube_dest.state
    return comparator.all()

In [12]:
# prints the output of the bidirectional_bfs
def print_info(visited_src, visited_dest, solution):
    print('solved cube!')
    discovered_nodes = len(visited_src) + len(visited_dest)
    path_len = len(solution) - 1
    print(f'Total number of nodes discovered: {discovered_nodes}')
    print(f'Length of the solution is: {path_len}')
    print(f"Solution is:\n")
    for sol in solution:
        print(f"\t{sol}")
    return discovered_nodes, path_len


def bidirect_bfs(cube):
    # we create the destination, the goal cube
    goal_cube = Cube(scrambled=False)
    
    if compare_states(cube, goal_cube):
        print('Cube already solved')
        return 0, 0
    
    # the queue where we add nodes discovered from the source
    queue_src = [cube.clone()]
    visited_src = {cube.hash(): [cube.hash()]}
 
    # the queue where we add nodes discovered from the source
    queue_dest = [goal_cube.clone()]
    visited_dest = {goal_cube.hash(): [goal_cube.hash()]}
    
    while queue_src and queue_dest:
        current_src = queue_src.pop(0)
        current_src_hash = current_src.hash()
        current_dest = queue_dest.pop(0)
        current_dest_hash = current_dest.hash()
        
        for neighbor in get_neighbours(current_src):
            probe_cube = Cube(scrambled=False)
            probe_cube.state = neighbor
            probe_cube_hash = probe_cube.hash()
            
            # we check if it's a new node
            if probe_cube_hash not in visited_src:
                # if we can find it in the destination queue, we have a path
                if probe_cube_hash in visited_dest:
                    solution = visited_src[current_src_hash] + visited_dest[probe_cube_hash][::-1]
                    return print_info(visited_src, visited_dest, solution)
                
                # if it's not, we add it in the source queue
                path_to_current_src = visited_src[current_src_hash] + [probe_cube_hash]
                visited_src[probe_cube_hash] = path_to_current_src
                queue_src.append(probe_cube)
        
        for neighbor in get_neighbours(current_dest):
            probe_cube = Cube(scrambled=False)
            probe_cube.state = neighbor
            probe_cube_hash = probe_cube.hash()
            
            # we check if it's a new node
            if probe_cube_hash not in visited_dest:
                # if we can find it in the destination queue, we have a path
                if probe_cube_hash in visited_src:
                    solution = visited_src[probe_cube_hash] + visited_dest[current_dest.hash()][::-1] 
                    return print_info(visited_src, visited_dest, solution)
                
                # if it's not, we add it in the source queue
                path_to_current_dest = visited_dest[current_dest.hash()] + [probe_cube_hash]
                visited_dest[probe_cube_hash] = path_to_current_dest
                queue_dest.append(probe_cube)
                    

### Let's see how well this algorithm performs

In [13]:
%%time
discovered_nodes_case1_bfs, path_len_case1_bfs = bidirect_bfs(cube1)

solved cube!
Total number of nodes discovered: 110
Length of the solution is: 5
Solution is:

	255011422234350044103135
	502511322234300144145035
	500111252232303444143550
	550021514242303440113352
	050511114242333340405252
	000011112222333344445555
CPU times: total: 0 ns
Wall time: 6.99 ms


In [14]:
%%time
discovered_nodes_case2_bfs, path_len_case2_bfs = bidirect_bfs(cube2)

solved cube!
Total number of nodes discovered: 782
Length of the solution is: 7
Solution is:

	115544133204352340002215
	515104033204322240351415
	512204513203320440351154
	125214113203350340052454
	105541114243350340032252
	051521214243330040415352
	001111222233330044445555
	000011112222333344445555
CPU times: total: 46.9 ms
Wall time: 39.4 ms


In [15]:
%%time
discovered_nodes_case3_bfs, path_len_case3_bfs = bidirect_bfs(cube3)

solved cube!
Total number of nodes discovered: 2719
Length of the solution is: 9
Solution is:

	311401145205353242254003
	321514013205353240254104
	323214153201350540250441
	343111545201350542220043
	303351412221350544210045
	333021112221300044554545
	330021302211302144554455
	303041402211352544230155
	003311002211332244445555
	000011112222333344445555
CPU times: total: 125 ms
Wall time: 126 ms


In [16]:
%%time
discovered_nodes_case4_bfs, path_len_case4_bfs = bidirect_bfs(cube4)

solved cube!
Total number of nodes discovered: 8303
Length of the solution is: 11
Solution is:

	521252101210333445030445
	541515023250333442020141
	540215503234331542021104
	510451052224331544021303
	511551042205332444020133
	512451152204330544023031
	145201352204320044513531
	140001522235320444513315
	401031322235310544052415
	004141222235350044331515
	000041412222353544331155
	000011112222333344445555
CPU times: total: 391 ms
Wall time: 379 ms
